# Simulations on a sheet domain

The following notebook contains sample code for simulating gas exchange on an infinite sheet geometry using the model outlined in Zurita & Hurtado (2022).

First we import necessary libraries and add important directories to the system path.

In [1]:
import sys
import os
import dolfin
import pyvista as pv
sys.path.append(os.getcwd()[:-6])
pv.set_jupyter_backend('ipyvtklink')

Following that, we import the main simulation class and parameter dictionary from the model source files.

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Now we outline the folder name and the path to save raw data files.

In [2]:
folder = "sheet-notebook"
path = os.path.join("../raw-data", folder)

Now we run the solver. First, we instance the model. Then, we generate the mesh that mimics an infinite sheet geometry. Afterwards we prescribe the blood velocity $\boldsymbol{u}$ and simulate the transport problem _without_ hemoglobin effects. Finally, we run the actual full model using the previous solution as an initial guess for the Newton iteration, and save the boundary data.

In [ ]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.generate_sheet_mesh(
    dims=(300, 6, 6), elems=(100, 5, 5), save=True, periodic=True, refined=True
)
model.set_u(value=(400, 0, 0), save=True)
x = model.sim_t(hb=False, save=False)
solution = model.sim_t(hb=True, save=True, guess=x)
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries

Now we use `PyVista` to visualize our results.

In [ ]:
plotter = pv.Plotter(shape=(2, 2))

plotter.subplot(0, 0)
plotter.add_text("Oxygen partial pressure")
filename = os.path.join(path, "t", "pO2000000.vtu")
reader = pv.get_reader(filename)
mesh = reader.read()
plotter.add_mesh(mesh, scalars="p_O2")

plotter.subplot(0, 1)
plotter.add_text("Carbon dioxide partial pressure")
filename = os.path.join(path, "t", "pCO2000000.vtu")
reader = pv.get_reader(filename)
mesh = reader.read()
plotter.add_mesh(mesh, scalars="p_CO2")

plotter.subplot(1, 0)
plotter.add_text("Oxyhemoglobin concentration")
filename = os.path.join(path, "t", "cHbO2000000.vtu")
reader = pv.get_reader(filename)
mesh = reader.read()
plotter.add_mesh(mesh, scalars="c_HbO2")

plotter.subplot(1, 1)
plotter.add_text("Carbaminohemoglobin concentration")
filename = os.path.join(path, "t", "cHbCO2000000.vtu")
reader = pv.get_reader(filename)
mesh = reader.read()
plotter.add_mesh(mesh, scalars="c_HbCO2")

plotter.show()

/home/pzurita/.local/lib/python3.10/site-packages/pyvista/plotting/plotting.py:131: UserWarning: 
This system does not appear to be running an xserver.
PyVista will likely segfault when rendering.

Try starting a virtual frame buffer with xvfb, or using
  ``pyvista.start_xvfb()``

  warnings.warn(
